# Python libraries and function definitions

In [1]:
import rdflib
from wikidataintegrator import wdi_core, wdi_login
from getpass import getpass
import pandas as pd
import sys
import traceback

def createProperty(login=login, wdprop=None, lulabel="", enlabel="", frlabel="", delabel="", description="", property_datatype=""):
    if wdprop== None:
        s = []
    else:
        s = [wdi_core.WDUrl(wdprop, prop_nr="P1")]
    localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(api,sparql)
    item = localEntityEngine(data=s)
    #item.set_label(lulabel, lang="lu")
    item.set_label(enlabel, lang="en")
    item.set_label(delabel, lang="de")
    item.set_label(frlabel, lang="fr")
    item.set_description(description, lang="en")
    
    print(item.write(login, entity_type="property", property_datatype=property_datatype))

# Import CIDOC-CRM definition
TODO: confirm that versions are same between DMSAF and the CIDOC-CRM definitions here

In [ ]:
cidoc = rdflib.Graph()
cidoc.load("https://raw.githubusercontent.com/erlangen-crm/ecrm/master/ecrm_current.owl", format="xml")

# Access details for the Wikibase which host the SAF

In [23]:
wbstack = "luxsafjun171"
wikibase = "https://{}.wiki.opencura.com/".format(wbstack)
api = "https://{}.wiki.opencura.com/w/api.php".format(wbstack)
sparql = "https://{}.wiki.opencura.com/query/sparql".format(wbstack)
entityUri = wikibase.replace("https:", "http:")+"entity/"

WBUSER = "Andrawaag"
WBPASS = getpass()
login = wdi_login.WDLogin(WBUSER, WBPASS, mediawiki_api_url=api)

········


# predefined properties from rdfs, skos, etc
 TODO work on multilangauge descriptions

In [4]:
# instance of
createProperty(login, lulabel="", 
                      enlabel="instance of",
                      frlabel="instance de",
                      delabel="ist ein(e)",
                      property_datatype="wikibase-item")

# subclass of
## TODO work on multilangauge descriptions 
## 
createProperty(login, lulabel="Ënnerklass vu(n)", 
                      enlabel="subclass of",
                      frlabel="sous-classe de",
                      delabel="Unterklasse von",
                      property_datatype="wikibase-item")
# skos:exact match
createProperty(login, lulabel="Ënnerklass vu(n)", 
                      enlabel="exact match",
                      frlabel="correspondance exacte",
                      delabel="exakte Übereinstimmung",
                      description="mapping",
                      property_datatype="url")
#domain 
createProperty(login, lulabel="", 
                      enlabel="domain",
                      frlabel="domaine",
                      delabel="domain",
                      property_datatype="wikibase-item")
#range
createProperty(login, lulabel="", 
                      enlabel="range",
                      frlabel="intervalle",
                      delabel="reichweite",
                      property_datatype="wikibase-item")
#subPropertyOf
createProperty(login, lulabel="", 
                      enlabel="subproperty of",
                      frlabel="sous-propriété de",
                      delabel="untereigenschaft von",
                      property_datatype="wikibase-item")
#inverseOf
createProperty(login, lulabel="", 
                      enlabel="inverse of",
                      frlabel="inverse de",
                      delabel="invers von",
                      property_datatype="wikibase-item")
propertyID = dict()
query = """SELECT ?prop ?propLabel WHERE {
  ?prop wikibase:directClaim ?wdt .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}"""
for index, row in wdi_core.WDItemEngine.execute_sparql_query(query, as_dataframe = True, endpoint=sparql).iterrows():
    print(row["prop"].replace(entityUri, ""), row["propLabel"])
    propertyID[row["propLabel"]] = row["prop"].replace(entityUri, "")

# class item
localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(api,sparql)
item = localEntityEngine(new_item=True)
item.set_label("Class", lang="en")
item.set_aliases(["Owl:Class"], lang="en")
item.write(login)

# property item
localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(api,sparql)
item = localEntityEngine(new_item=True)
item.set_label("Property", lang="en")
item.set_aliases(["owl:ObjectProperty"], lang="en")
item.write(login)
propertyID

/usr/local/lib/python3.9/site-packages/wikidataintegrator/wdi_core.py:485: UserWarning: Warning: No distinct value properties found
Please set P2302 and Q21502410 in your wikibase or set `core_props` manually.
Continuing with no core_props
  warnings.warn("Warning: No distinct value properties found\n" +


P1


P1 instance of
P2 subclass of
P3 exact match
P4 domain
P5 range
P6 subproperty of
P7 inverse of


{'instance of': 'P1',
 'subclass of': 'P2',
 'exact match': 'P3',
 'domain': 'P4',
 'range': 'P5',
 'subproperty of': 'P6',
 'inverse of': 'P7'}

# Create CIDOC-CRM items on WB SAF

In [20]:
query = "SELECT ?cidoc ?label WHERE {?cidoc rdf:type <http://www.w3.org/2002/07/owl#Class> ; rdfs:label ?label .}"
for row in cidoc.query(query):
    #print(str(row[0]), str(row[1]), " ".join(str(row[1]).split(" ")[1:]))
    statements=[]
    statements.append(wdi_core.WDItemID(value="Q1", prop_nr=propertyID["instance of"]))
    statements.append(wdi_core.WDUrl(value=str(row[0]), prop_nr=propertyID['exact match']))
    item = localEntityEngine(new_item=True, data=statements)
    item.set_label(" ".join(str(row[1]).split(" ")), lang="en")
    #item.set_aliases([str(row[1]), str(row[1]).replace(" ", "_")], lang="en")
    print(item.write(login))

#propertyClasses
query = "SELECT ?cidoc ?label WHERE {?cidoc rdf:type <http://www.w3.org/2002/07/owl#ObjectProperty> ; rdfs:label ?label .}"
for row in cidoc.query(query):
    print(str(row[0]), str(row[1]), " ".join(str(row[1]).split(" ")[1:]))
    statements=[]
    statements.append(wdi_core.WDItemID(value="Q2", prop_nr=propertyID["instance of"]))
    statements.append(wdi_core.WDUrl(value=str(row[0]), prop_nr=propertyID["exact match"]))
    item = localEntityEngine(new_item=True, data=statements)
    item.set_label(" ".join(str(row[1]).split(" ")), lang="en")
    #item.set_aliases([str(row[1]), str(row[1]).replace(" ", "_")], lang="en")
    try:
        print(item.write(login))
    except:
        print("ERROR: "+str(row[1]))
        continue

qid = dict()
query = "PREFIX wdt: <http://{}.wiki.opencura.com/prop/direct/> SELECT * WHERE {{?item wdt:P3 ?uri .}}".format(wbstack)
for index, row in wdi_core.WDItemEngine.execute_sparql_query(query, as_dataframe = True, endpoint=sparql).iterrows():
    qid[row["uri"]] =row["item"].replace(entityUri, "")

Q3
Q4
Q5
Q6
Q7
Q8
Q9
Q10
Q11
Q12
Q13
Q14
Q15
Q16
Q17
Q18
Q19
Q20
Q21
Q22
Q23
Q24
Q25
Q26
Q27
Q28
Q29
Q30
Q31
Q32
Q33
Q34
Q35
Q36
Q37
Q38
Q39
Q40
Q41
Q42
Q43
Q44
Q45
Q46
Q47
Q48
Q49
Q50
Q51
Q52
Q53
Q54
Q55
Q56
Q57
Q58
Q59
Q60
Q61
Q62
Q63
Q64
Q65
Q66
Q67
Q68
Q69
Q70
Q71
Q72
Q73


In [29]:

query = """SELECT DISTINCT ?cidoc ?domain  WHERE {
                 ?cidoc rdf:type <http://www.w3.org/2002/07/owl#ObjectProperty> ; 
                        rdfs:domain ?domain .}"""
for row in cidoc.query(query):
    try:
        print(qid[str(row["cidoc"])].replace(entityUri, ""))
        localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(api,sparql)
        statements=[]
        statements.append(wdi_core.WDItemID(value=qid[str(row["domain"])], prop_nr=propertyID["domain"]))
        item = localEntityEngine(wd_item_id=qid[str(row["cidoc"])], data=statements)
        print(item.write(login))
    except Exception as e:
        print(traceback.format_exc())
        continue





Traceback (most recent call last):
  File "<ipython-input-29-ca45696dead3>", line 8, in <module>
    print(qid[str(row["cidoc"])].replace(entityUri, ""))
KeyError: 'http://erlangen-crm.org/current/P104_is_subject_to'

Q74
Q74
Q75
Q75
Q77
Q77
Q78
Q78
Q79
Q79
Q80
Q80
Q81
Q81
Q82
Q82
Traceback (most recent call last):
  File "<ipython-input-29-ca45696dead3>", line 8, in <module>
    print(qid[str(row["cidoc"])].replace(entityUri, ""))
KeyError: 'http://erlangen-crm.org/current/P167i_was_place_of'

Q85
Q85
Q86
Q86
Q88
Traceback (most recent call last):
  File "<ipython-input-29-ca45696dead3>", line 11, in <module>
    statements.append(wdi_core.WDItemID(value=qid[str(row["domain"])], prop_nr=propertyID["domain"]))
KeyError: 'http://erlangen-crm.org/current/E99_Product_Type'

Q89
Q89
Q90
Q90
Q91
Q91
Q92
Q92
Q93
Q93
Q94
Q94
Q95
Q95
Q96
Q96
Q97
Q97
Q98
Q98
Q99
Q99
Q100
Q100
Q101
Q101
Q102
Q102
Q103
Q103
Traceback (most recent call last):
  File "<ipython-input-29-ca45696dead3>", line 8, in <m

# Add CIDOC-CRM relations
## Domain

In [30]:
query = """SELECT DISTINCT ?cidoc ?range  WHERE {
                 ?cidoc rdf:type <http://www.w3.org/2002/07/owl#ObjectProperty> ; 
                        rdfs:range ?range .}"""
for row in cidoc.query(query):
    try:
        print(qid[str(row["cidoc"])])
        localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(api,sparql)
        statements=[]
        statements.append(wdi_core.WDItemID(value=qid[str(row["range"])].replace(entityUri, ""), prop_nr=propertyID["range"]))
        item = localEntityEngine(wd_item_id=qid[str(row["cidoc"])].replace(entityUri, ""), data=statements)
        print(item.write(login))
    except:
        str(row["cidoc"])
        continue

Q74
Q74
Q75
Q75
Q76
Q76
Q77
Q77
Q78
Q78
Q79
Q79
Q80
Q80
Q81
Q81
Q82
Q82
Q83
Q83
Q84
Q84
Q85
Q85
Q86
Q86
Q87
Q87
Q88
Q88
Q90
Q90
Q91
Q91
Q92
Q92
Q93
Q93
Q94
Q94
Q95
Q95
Q96
Q96
Q97
Q97
Q98
Q98
Q99
Q99
Q100
Q100
Q101
Q102
Q102
Q103
Q103
Q104
Q104
Q105
Q105
Q106
Q106
Q107
Q107
Q108
Q108
Q109
Q109
Q110
Q110
Q111
Q111
Q112
Q112
Q113
Q113
Q114
Q114
Q115
Q115
Q116
Q116
Q117
Q117
Q118
Q118
Q119
Q119
Q120
Q120
Q121
Q121
Q122
Q122
Q123
Q123
Q124
Q124
Q125
Q125
Q126
Q126
Q127
Q127
Q128
Q128
Q129
Q129
Q130
Q130
Q131
Q131
Q132
Q132
Q133
Q133
Q134
Q134
Q135
Q135
Q136
Q136
Q137
Q137
Q139
Q139
Q140
Q140
Q141
Q142
Q142
Q143
Q143
Q144
Q144
Q145
Q145
Q146
Q146
Q147
Q147
Q148
Q148
Q149
Q149
Q150
Q150
Q151
Q151
Q152
Q152
Q153
Q153
Q154
Q154
Q155
Q155
Q156
Q156
Q157
Q157
Q158
Q158
Q159
Q159
Q160
Q160
Q161
Q161
Q164
Q164
Q165
Q165
Q166
Q166
Q167
Q167
Q168
Q168
Q172
Q172
Q173
Q173
Q174
Q174
Q175
Q175
Q176
Q176
Q177
Q177
Q178
Q178
Q179
Q179
Q180
Q180
Q181
Q181
Q182
Q182
Q183
Q183
Q184
Q184
Q185
Q185
Q186
Q186


In [34]:
## SubPropertyOf

In [31]:
query = """SELECT DISTINCT ?cidoc ?subPropertyOf  WHERE {
                 ?cidoc rdf:type <http://www.w3.org/2002/07/owl#ObjectProperty> ; 
                        rdfs:subPropertyOf ?subPropertyOf ;
                        rdfs:range ?range .}"""
for row in cidoc.query(query):
    try:
        print(qid[str(row["cidoc"])])
        localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(api,sparql)
        statements=[]
        statements.append(wdi_core.WDItemID(value=qid[str(row["subPropertyOf"])], prop_nr=propertyID["subproperty of"]))
        item = localEntityEngine(wd_item_id=qid[str(row["cidoc"])], data=statements)
        print(item.write(login))
    except:
        continue

Q74
Q74
Q74
Q74
Q77
Q77
Q79
Q79
Q79
Q79
Q80
Q80
Q82
Q82
Q83
Q83
Q84
Q84
Q85
Q85
Q86
Q86
Q87
Q87
Q94
Q94
Q95
Q95
Q96
Q96
Q97
Q97
Q102
Q102
Q104
Q104
Q105
Q105
Q106
Q106
Q107
Q107
Q108
Q108
Q109
Q109
Q111
Q111
Q113
Q113
Q114
Q114
Q116
Q116
Q118
Q118
Q120
Q120
Q121
Q121
Q123
Q123
Q124
Q124
Q125
Q125
Q127
Q127
Q128
Q128
Q129
Q129
Q130
Q130
Q133
Q133
Q135
Q135
Q137
Q137
Q139
Q139
Q144
Q144
Q146
Q146
Q149
Q149
Q151
Q151
Q152
Q152
Q154
Q154
Q158
Q158
Q160
Q160
Q161
Q161
Q164
Q164
Q165
Q165
Q166
Q166
Q168
Q168
Q174
Q174
Q174
Q174
Q177
Q177
Q178
Q178
Q181
Q181
Q182
Q182
Q183
Q183
Q186
Q186
Q189
Q189
Q189
Q189
Q190
Q190
Q191
Q191
Q194
Q194
Q195
Q195
Q195
Q195
Q197
Q197
Q198
Q198
Q200
Q200
Q202
Q202
Q204
Q204
Q205
Q205
Q207
Q207
Q211
Q211
Q212
Q212
Q215
Q215
Q215
Q215
Q216
Q216
Q219
Q219
Q219
Q219
Q220
Q220
Q221
Q221
Q222
Q222
Q223
Q223
Q225
Q225
Q226
Q226
Q229
Q229
Q232
Q232
Q234
Q234
Q237
Q237
Q238
Q238
Q245
Q245
Q247
Q247
Q248
Q248
Q248
Q248
Q249
Q249
Q251
Q251
Q254
Q254
Q255
Q255
Q258
Q258
Q2

In [35]:
## InverseOf

In [32]:
query = """SELECT DISTINCT ?cidoc ?inverseOf  WHERE {
                 ?cidoc rdf:type <http://www.w3.org/2002/07/owl#ObjectProperty> ; 
                        owl:inverseOf ?inverseOf ;
                        rdfs:range ?range .}"""
for row in cidoc.query(query):
    try:
        print(qid[str(row["cidoc"])])
        localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(api,sparql)
        statements=[]
        statements.append(wdi_core.WDItemID(value=qid[str(row["inverseOf"])], prop_nr=propertyID["inverse of"]))
        item = localEntityEngine(wd_item_id=qid[str(row["cidoc"])], data=statements)
        print(item.write(login))
    except:
        continue

Q74
Q74
Q75
Q76
Q76
Q77
Q77
Q78
Q78
Q79
Q79
Q80
Q80
Q81
Q81
Q82
Q82
Q83
Q83
Q84
Q84
Q85
Q85
Q86
Q87
Q87
Q88
Q90
Q90
Q91
Q91
Q92
Q92
Q93
Q93
Q94
Q94
Q95
Q95
Q96
Q96
Q97
Q97
Q98
Q98
Q99
Q99
Q100
Q100
Q101
Q101
Q102
Q102
Q103
Q103
Q104
Q104
Q105
Q105
Q106
Q106
Q107
Q107
Q108
Q108
Q109
Q109
Q110
Q110
Q111
Q111
Q112
Q112
Q113
Q113
Q114
Q114
Q115
Q115
Q116
Q116
Q117
Q117
Q118
Q118
Q119
Q119
Q120
Q120
Q121
Q121
Q122
Q122
Q123
Q123
Q124
Q124
Q125
Q125
Q126
Q126
Q127
Q127
Q128
Q128
Q129
Q129
Q130
Q130
Q131
Q131
Q132
Q132
Q133
Q133
Q134
Q134
Q135
Q135
Q136
Q136
Q137
Q137
Q139
Q139
Q140
Q140
Q141
Q142
Q142
Q143
Q143
Q144
Q144
Q145
Q145
Q146
Q146
Q147
Q147
Q148
Q148
Q149
Q149
Q150
Q150
Q151
Q151
Q152
Q152
Q153
Q154
Q154
Q155
Q155
Q156
Q156
Q157
Q157
Q158
Q158
Q159
Q159
Q160
Q160
Q161
Q161
Q164
Q164
Q165
Q165
Q166
Q166
Q167
Q167
Q168
Q168
Q172
Q172
Q173
Q173
Q174
Q174
Q175
Q175
Q176
Q176
Q177
Q177
Q178
Q178
Q179
Q179
Q180
Q180
Q181
Q181
Q182
Q182
Q183
Q183
Q184
Q184
Q185
Q185
Q186
Q187
Q187
Q188
Q1